In [12]:
#Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2

In [2]:
train_data_dir = './train'  # Path to your training dataset
test_data_dir = './test'

In [3]:
#Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
# Data Normalization
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Loading Training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 5181 images belonging to 5 classes.


In [6]:
# Loading Testing data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 236 images belonging to 5 classes.


In [7]:
#CNN architecture with 4 layers and upto 256 filters
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.5),
    
    GlobalAveragePooling2D(),  # This layer reduces dimensions without the need for Flatten
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])


C:\Users\NITHIN\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 124, 124, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 60, 60, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 58, 58, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 58, 58, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 27, 27, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 27, 27, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 25, 25, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 10, 10, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 10, 10, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 618,533 (2.36 MB)

 Trainable params: 617,125 (2.35 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [14]:
##### from tensorflow.keras.callbacks import ModelCheckpoint

# Define the callback to save the best model based on validation loss
checkpoint_callback = ModelCheckpoint(
    "./best_model.keras",                   # Path to save the best model
    monitor="val_loss",                 # Monitor validation loss
    save_best_only=True,                # Save only the best model
    mode="min",                         # Save when validation loss decreases
    verbose=1                           # Verbose output
)

# Train the model with the checkpoint callback
history = model.fit(
    train_generator,
    epochs=101,
    validation_data=test_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=test_generator.samples // 32,
    callbacks=[checkpoint_callback]     # Add the checkpoint callback
)


Epoch 1/101


C:\Users\NITHIN\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 0.2663 - loss: 3.4593
Epoch 1: val_loss improved from inf to 2.26519, saving model to ./best_model.keras
161/161 ━━━━━━━━━━━━━━━━━━━━ 57s 309ms/step - accuracy: 0.2664 - loss: 3.4558 - val_accuracy: 0.2321 - val_loss: 2.2652
Epoch 2/101
  1/161 ━━━━━━━━━━━━━━━━━━━━ 38s 239ms/step - accuracy: 0.3750 - loss: 2.3297
Epoch 2: val_loss did not improve from 2.26519
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step - accuracy: 0.3750 - loss: 2.3297 - val_accuracy: 0.2500 - val_loss: 2.3392
Epoch 3/101


C:\Users\NITHIN\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.2971 - loss: 2.3638
Epoch 3: val_loss improved from 2.26519 to 2.22386, saving model to ./best_model.keras
161/161 ━━━━━━━━━━━━━━━━━━━━ 48s 295ms/step - accuracy: 0.2972 - loss: 2.3635 - val_accuracy: 0.3482 - val_loss: 2.2239
Epoch 4/101
  1/161 ━━━━━━━━━━━━━━━━━━━━ 55s 346ms/step - accuracy: 0.4062 - loss: 2.2829
Epoch 4: val_loss improved from 2.22386 to 2.22283, saving model to ./best_model.keras
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4062 - loss: 2.2829 - val_accuracy: 0.3333 - val_loss: 2.2228
Epoch 5/101
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 0.3406 - loss: 2.2640
Epoch 5: val_loss did not improve from 2.22283
161/161 ━━━━━━━━━━━━━━━━━━━━ 52s 316ms/step - accuracy: 0.3406 - loss: 2.2639 - val_accuracy: 0.3170 - val_loss: 2.3421
Epoch 6/101
  1/161 ━━━━━━━━━━━━━━━━━━━━ 41s 259ms/step - accuracy: 0.2812 - loss: 2.2457
Epoch 6: val_loss improved from 2.22283 to 2.19486, saving model to ./best_m